In [ ]:
# see link
# https://github.com/Fraud-Detection-Handbook

package ='07-sklearn.ensemble'
name='Bagging'
tuningAndParameters='02-After tuning'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import datamanagement as dm
reload(dm)

import result as resultMd
reload(resultMd)

import graph as gf
reload(gf)

print('done')

In [ ]:
dfLearning, dfValidation =dm.getDataLearningAndValidation()

dfLearning.head()

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0

predictors = dm.getPredictors(dfLearning)
target = dm.getTarget()

x_train, x_test, y_train, y_test = train_test_split(dfLearning[predictors], dfLearning[target], test_size = TEST_SIZE, 
                                                        stratify=dfLearning[target],
                                                        random_state = RANDOM_STATE)

In [ ]:
%%script false

from sklearn.ensemble import BaggingClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

dic_param={
 'n_estimators': randint(50,170)
}
modelClf = BaggingClassifier(random_state=42)

random_search = RandomizedSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)

#{'n_estimators': 139}
#0.533043533043533

#{'n_estimators': 66}
#0.5391547049441786


In [ ]:
#%%script false

from sklearn.ensemble import BaggingClassifier
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV

dic_param={
    'n_estimators': [65,70,75,80,85]
}
modelClf = BaggingClassifier(random_state=42)

random_search = GridSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)

#{'n_estimators': 130}
#0.533043533043533

#{'n_estimators': 66}
#0.5391547049441786

#{'n_estimators': 75}
#0.5430008587903324


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier
from datetime import datetime

modelClf = BaggingClassifier(random_state=42)
parameters={ 'n_estimators': 75}

modelClf.set_params(**parameters)
then= datetime.now()
modelClf.fit(x_train, y_train)
now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result(package, name, tuningAndParameters, duration_in_s)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

dm.show_confusion_matrix(y_train, predsTrain,'Confusion matrix learning data')
print(f"f1 train {f1_score(y_train, predsTrain):.3f}")
dm.show_confusion_matrix(y_test, predsTest,'Confusion matrix test data')
print(f"f1 test {f1_score(y_test, predsTest):.3f}")

In [ ]:
gf.show_importance(modelClf, predictors)

In [ ]:
gf.show_prediction_graph(modelClf, x_test,y_test)

In [ ]:
predsValidation = modelClf.predict(dfValidation[predictors])
f1=f1_score(dfValidation[target], predsValidation)

dm.show_confusion_matrix(dfValidation[target], predsValidation,'Confusion matrix validation data')
print(f"f1 validation {f1:.3f}")
resultMd.update_performance_result(package, name, tuningAndParameters, f1)